In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

df = pd.read_csv('random_forest_dirty_dataset.csv')


In [24]:
numerical_cols = df.select_dtypes(include='number').columns.drop('target').tolist()
categorical_cols = df.select_dtypes('object').columns.tolist()

In [39]:
numerical_pipline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_pipline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('numerical_cols', numerical_pipline, numerical_cols),
    ('categorical_cols', categorical_pipline, categorical_cols)
])

In [40]:
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [41]:
param_distributions = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

In [42]:
search = RandomizedSearchCV(
    estimator=full_pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    verbose=1,
    random_state=42,
    n_jobs=-1
)

In [37]:
X = df.drop(columns='target')
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [43]:
search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('numerical_cols',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               ['feature_0',
                                                                                'feature_1',
                                                                                'feature_2',
                                                                                'feature_3',
                                                                                'feature_4',
                                                                                'feature_5',
                                                                                'feature_6',
                                                                                'feature_7',
                                                                                'feature_8',
                                                                                'feature_9']),
                                                                              ('categorical_cols',
                                                                               Pipeline(steps=[('imputer',
                                                                                                S...
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                                                                               ['category_1',
                                                                                'category_2'])])),
                                             ('classifier',
                                              RandomForestClassifier(random_state=42))]),
                   n_jobs=-1,
                   param_distributions={'classifier__max_depth': [None, 10, 20,
                                                                  30],
                                        'classifier__min_samples_leaf': [1, 2,
                                                                         4],
                                        'classifier__min_samples_split': [2, 5,
                                                                          10],
                                        'classifier__n_estimators': [100, 200,
                                                                     300]},
                   random_state=42, scoring='accuracy', verbose=1)